In [1]:
import logging
log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 15
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

import os
from dotenv import load_dotenv
load_dotenv()
secret_name = "OPENAI_API_KEY_GDPR"
openai_api_key = os.getenv(secret_name)
key = os.getenv('DECRYPTION_KEY_GDPR')


In [2]:
import os
import pandas as pd
from openai import OpenAI

from regulations_rag.embeddings import  EmbeddingParameters
from regulations_rag.rerank import RerankAlgos
from regulations_rag.corpus_chat import ChatParameters

import importlib
import gdpr_rag.corpus_index
importlib.reload(gdpr_rag.corpus_index)
from gdpr_rag.corpus_index import GDPRCorpusIndex

import regulations_rag.corpus_chat
importlib.reload(regulations_rag.corpus_index)
from regulations_rag.corpus_chat import CorpusChat

from IPython.display import Markdown, display

# import gdpr_rag.corpus_chat
# importlib.reload(gdpr_rag.corpus_chat)
# from gdpr_rag.corpus_chat import CorpusChatCEMAD


In [3]:
api_key=os.environ.get("OPENAI_API_KEY")
chat_parameters = ChatParameters(chat_model = "gpt-4o", api_key=api_key, temperature = 0, max_tokens = 500, token_limit_when_truncating_message_queue = 3500)
#chat_parameters = ChatParameters(chat_model = "gpt-4-turbo", temperature = 0, max_tokens = 500)

embedding_parameters = EmbeddingParameters("text-embedding-3-large", 1024)

corpus_index = GDPRCorpusIndex(key)

rerank_algo  = RerankAlgos.LLM
if rerank_algo == RerankAlgos.LLM:
    rerank_algo.params["openai_client"] = chat_parameters.openai_client
    rerank_algo.params["model_to_use"] = chat_parameters.model
    rerank_algo.params["user_type"] = corpus_index.user_type
    rerank_algo.params["corpus_description"] = corpus_index.corpus_description
    rerank_algo.params["final_token_cap"] = 5000 # can go large with the new models


chat = CorpusChat(
                    embedding_parameters = embedding_parameters, 
                    chat_parameters = chat_parameters, 
                    corpus_index = corpus_index,
                    rerank_algo = RerankAlgos.LLM,   
                    user_name_for_logging = 'test_user')

chat.strick_rag = False



DEV:regulations_rag.corpus_chat:test_user: Reset Conversation History


In [11]:

#user_content = "Can I collect data from children?"
user_content = "What is required from you to ensure data protection by design and by default?"
user_content = "How can you show that you comply with data protection by design and by default requirements?"
user_content = "What should processors and producers consider when helping you design or procure solutions that embed data protection by design and by default (DPbDD)?"
user_content = "What is the process for reporting changes in binding corporate rules?"
user_content = "Can special categories of personal data be used in automated decision-making?"

# user_content = "what is the difference between a controller and a dpo?"
# user_content = "Write me what criteria in terms of the GDPR regulation must be met by the controller of personal data that operates CCTV?"
# user_content = "How does GDPR relate to ISO27001 and the AI ​​Act?"
#user_content = "How does GDPR relate to ISO27001 and the AI Act?"

# user_content = "Can an employer monitor employees?"
# user_content = "hi"
user_content = "Can employers monitor workers?"
#user_content = "Is video surveillance regulated"
user_content = "what is gdpr?"

chat.strict_rag = False
chat.reset_conversation_history()
chat.user_provides_input(user_content)

DEV:regulations_rag.corpus_chat:test_user: Reset Conversation History
DEV:regulations_rag.corpus_chat:CorpusChat.user_provides_input() processing test_user's question: what is gdpr?
DEV:regulations_rag.path_search:similarity_search called
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.0531:                 GDPR:                    2:        question: What is gdpr?
DEV:regulations_rag.corpus_index:0.3627:                 GDPR:                    2:        question: What does GDPR apply to?
DEV:regulations_rag.corpus_index:--   Relevant sections found
DEV:regulations_rag.rerank:Re-ranking using LLM
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEV:regulations_rag.rerank:--   results requested by LLM filter
DEV:reg

In [8]:
display(Markdown(chat.messages_intermediate[-1]['content']))

The system was not able to answer the question using the provided documents

In [6]:
user_content = 'Can you summarise Section 3 from General Data Protection Regulation?'
chat.strick_rag = False
chat.user_provides_input(user_content)
display(Markdown(chat.messages_intermediate[-1]['content']))

DEV:regulations_rag.corpus_chat:CorpusChat.user_provides_input() processing test_user's question: Can you summarise Section 3 from General Data Protection Regulation?
DEV:regulations_rag.path_search:similarity_search called
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.3676:          CovidHealth:                    6:        question:  Are there any built-in restrictions on individuals' rights under GDPR (General Data Protection Regulation)?
DEV:regulations_rag.corpus_index:--   Relevant sections found
DEV:regulations_rag.rerank:Re-ranking using LLM
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEV:regulations_rag.rerank: should be an integer between 1 and 1 but it is not
DEV:regulations_rag.rerank:--   results

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


NOTE: The following answer is provided without references and should therefore be treated with caution.

Section 3 of the General Data Protection Regulation (GDPR) outlines the territorial scope of the regulation. It specifies that the GDPR applies to the processing of personal data in the context of activities of an establishment of a controller or processor in the EU, regardless of where the processing occurs. It also applies to non-EU entities that process personal data of individuals in the EU when offering goods or services to them or monitoring their behavior within the EU. Additionally, it applies to processing by a controller not established in the EU but in a place where EU Member State law applies by virtue of public international law.

In [11]:
raw_response



{'role': 'assistant',
 'content': "Special categories of personal data can be used in automated decision-making only under specific conditions. According to Article 22(4), such processing is allowed if there is an applicable Article 22(2) exemption, and point (a) or (g) of Article 9(2) applies. This means that the processing must either be based on the explicit consent of the data subject (Article 9(2)(a)) or be necessary for reasons of substantial public interest, on the basis of Union or Member State law (Article 9(2)(g)). In both cases, suitable measures must be in place to safeguard the data subject's rights and freedoms and legitimate interests. \nReference: \nSection 22 from General Data Protection Regulation: \n# 22 Automated individual decision-making, including profiling\n\n&nbsp;&nbsp;&nbsp;&nbsp;1. The data subject shall have the right not to be subject to a decision based solely on automated processing, including profiling, which produces legal effects concerning him or her

In [6]:
from IPython.display import Markdown, display

m = chat.messages[-1]["content"].strip()
print(chat.messages[-1]["content"])

references = chat.references[m]
for index, row in references.iterrows():
    display(Markdown(row['text']))    


AttributeError: 'CorpusChat' object has no attribute 'messages'

In [9]:
chat.references

{'Processors and producers should consider several key factors when helping controllers design or procure solutions that embed data protection by design and by default (DPbDD):\n\n1. **Expert Guidance**: Use their expertise to build trust and guide controllers, including SMEs, in designing or procuring solutions that embed data protection into the processing.\n2. **Facilitate Controllers\' Needs**: Ensure that the design of products and services facilitates the controllers\' needs.\n3. **Effective Implementation**: Focus on the effective implementation of the principles and protection of the rights of data subjects.\n4. **Certification**: Strive to demonstrate DPbDD in the lifecycle of their development of a processing solution and consider certification as it provides added value and competitive advantage.\n5. **Specific Protection**: Provide specific protection for children under 18 and other vulnerable groups.\n6. **Support Compliance**: Facilitate DPbDD implementation to support th

In [4]:
from IPython.display import Markdown, display

# m = chat.corpus.get_text("Forgotten", "1")
# display(Markdown(m))    

d = chat.corpus.get_document("Forgotten")
d
#print(chat.corpus.get_text("CovidLocation", "1"))
#print(chat.corpus.get_text("Consent", "7.1.3").strip())
#print(chat.corpus.get_text("GDPR", "8").strip())

In [6]:
d.get_text("3.4")

"3 PART 3 - APPLICABILITY OF ARTICLE 6(1)(B) IN SPECIFIC SITUATIONS \n\n3.4 Processing for personalisation of content[^34]\n57. The EDPB acknowledges that personalisation of content may (but does not always) constitute an intrinsic and expected element of certain online services, and therefore may be regarded as necessary for the performance of the contract with the service user in some cases. Whether such processing can be regarded as an intrinsic aspect of an online service, will depend on the nature of the service provided, the expectations of the average data subject in light not only of the terms of service but also the way the service is promoted to users, and whether the service can be provided without personalisation. Where personalisation of content is not objectively necessary for the purpose of the underlying contract, for example where personalised content delivery is intended to increase user engagement with a service but is not an integral part of using the service, data 

In [33]:
from gdpr_rag.documents.transparency import Transparency
t = Transparency()
t.get_text("I")

'I Restrictions on data subject rights \n68. Article 23 provides for Member States (or the EU) to legislate for further restrictions on the scope of the data subject rights in relation to transparency and the substantive data subject rights[^55] where such measures respect the essence of the fundamental rights and freedoms and are necessary and proportionate to safeguard one or more of the ten objectives set out in Article 23.1(a) to (j). Where such national measures lessen either the specific data subject rights or the general transparency obligations, which would otherwise apply to data controllers under the GDPR, the data controller should be able to demonstrate how the national provision applies to them. As set out in Article 23.2(h), the legislative measure must contain a provision as to the right of the data subject to be informed about a restriction on their rights, unless so informing them may be prejudicial to the purpose of the restriction. Consistent with this, and in line w

In [5]:
print(chat._create_system_message(3))

You are answering questions about the General Data Protection Regulation (GDPR) for a Controller based only on the reference extracts provided. You have 3 options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. All referenced extracts must be quoted at the end of the answer, not in the body, by number, in a comma separated list starting after the keyword 'Reference: '. Do not include the word Extract, only provide the number(s).
2) Request additional documentation. If, in the body of the extract(s) provided, there is a reference to another section that is directly relevant and not already provided, respond with the word 'SECTION:' followed by 'Extract extract_number, Reference section_reference' - for example SECTION: Extract 1, Reference Article 98.
3) State 'NONE:' and nothing else in all other cases



In [ ]:
# BCR
user_content = "What are binding corporate rules?"
user_content = "How do binding corporate rules become approved?"
user_content = "What requirements must binding corporate rules meet?"
user_content = "What details must be spelled out in binding corporate rules?"
user_content = "What rights do individuals have under binding corporate rules?"
user_content = "Who is liable if binding corporate rules are breached?"
user_content = "How are individuals informed about their rights under binding corporate rules?"
user_content = "What is the role of a data protection officer within a group adopting binding corporate rules?"
user_content = "What mechanisms ensure compliance with binding corporate rules?"
user_content = "How is the effectiveness of binding corporate rules verified?"
user_content = "What happens if local laws conflict with the standards of binding corporate rules?"
user_content = "What kind of training is required for employees with access to personal data under binding corporate rules?"
user_content = "What is the process for reporting changes in binding corporate rules?"

# Decision Making
user_content = "What rights do individuals have regarding automated decision-making and profiling?"
user_content = "What does it mean for a decision to have a legal effect on a person?"
user_content = "Under what conditions can an individual be subject to automated decision-making?"
user_content = "What safeguards must be in place when automated decisions are necessary for contracts?"
user_content = "Can special categories of personal data be used in automated decision-making?"

#DPIA
user_content = "When must you conduct a data protection impact assessment?"
user_content = "Who should you consult when carrying out a data protection impact assessment?"
user_content = " What triggers the requirement for a data protection impact assessment?"
user_content = "How do supervisory authorities contribute to the DPIA process?"
user_content = "How do supervisory authorities contribute to the data protection impact assessment process?"
user_content ="What content is required in a data protection impact assessment?"
user_content ="What content is required in a DPIA?"
user_content = "Should individuals' views be considered in the assessment process?"
user_content = "How often should you review the data protection impact assessment?"

#DPO
user_content = "When must you designate a data protection officer?"
user_content = "Can multiple organisations share a DPO?"
user_content = "What criteria must a data protection officer meet?"
user_content = "Can a data protection officer be a contractor?"
user_content = "How should the contact details of the DPO be shared?"
user_content = "What types of processing activities require you to appoint a data protection officer?"
user_content = "How does the scale of data processing affect the requirement to appoint a data protection officer?"
user_content = "What is large scale?"
user_content = "What is the difference between a data protection officer and a controller?"

# intl_transfer
user_content = "What are the conditions for transferring personal data to another country or international organisation?"
user_content = "When is an individual's consent required for data transfer?"
user_content = "How does a contract affect the transfer of personal data?"
user_content = "What constitutes an important reason of public interest for transferring personal data?"
user_content = "When can data be transferred for the establishment, exercise, or defence of legal claims?"
user_content = "Under what circumstances can personal data be transferred to protect someone's vital interests?"
user_content = "What are the rules for transferring data from public registers?"
user_content = "What are the requirements for a non-repetitive transfer based on compelling legitimate interests?"
user_content = "When are public authorities exempt from certain transfer conditions?"
user_content = "How does the law recognise public interest for data transfers?"
user_content = "What documentation is required for assessing data transfers?"

# lead_sa

# data breach
user_content = "What action should you take if a personal data breach occurs?"
user_content = "How soon must you notify the appropriate authority when a data breach is discovered?"
user_content = "Is there any situation where you might not need to notify a supervisory authority about a data breach?"
user_content = "What information must you include when reporting a data breach?"
user_content = "What should be done if all details about a breach can't be reported immediately?"
user_content = "Why is it important to document all personal data breaches and the response to them?"

user_content = "When should you inform individuals about a personal data breach?"
user_content = "What information should be included when notifying an individual about a personal data breach?"
user_content = "Under what conditions can you avoid informing individuals about a personal data breach?"
user_content = "What happens if you don't voluntarily notify individuals about a personal data breach?"


# portability
user_content = "What is the right to data portability?"
user_content = "What does data portability mean for the transfer of data between controllers?"
user_content = "Is the right to data portability unlimited?"

# transparency
user_content = "What information must be provided to individuals when collecting their personal data?"
user_content = "What details about the data controller need to be disclosed during data collection?"
user_content = "What are the required disclosures about the use and processing of personal data?"
user_content = "When is it necessary to inform individuals about the transfer of their personal data to third countries or international organisations?"
user_content = "What rights must individuals be informed about concerning their personal data?"
user_content = "How does the requirement for providing information change if the purpose of data processing changes?"
user_content = "What information must be provided to individuals when their data is collected indirectly?"
user_content = "Within what timeframe must information be provided to the data subject when their data is collected indirectly?"
# user_content = "What actions must be taken if the purpose of data processing changes after initial collection?"
# user_content = "Are there any circumstances under which the obligation to provide information to data subjects does not apply?"
# user_content = "How are data subjects informed about their rights concerning their personal data?"


# Codes
user_content = "Who is encouraged to create codes of conduct for GDPR compliance?"
user_content = "What purposes do codes of conduct serve?"
user_content = "Can codes of conduct address the protection of children's data?"
user_content = "How can codes of conduct assist with personal data breaches?"
user_content = "Can organisations outside the EU adhere to codes of conduct?"
user_content = "What requirements must a code of conduct meet for approval?"
# user_content = "Who approves codes of conduct for local purposes?"
# user_content = "How are codes of conduct evaluated for processing activities across multiple Member States?"
# user_content = "What role does the Commission play in the validity of codes of conduct?"
# user_content = "How is the public informed about approved codes of conduct?"
# user_content = "Where can one find a register of approved codes of conduct?"
# user_content = "Who can monitor compliance with a code of conduct?"
# user_content = "What qualifications must a body meet to monitor compliance with a code of conduct?"
user_content = "How does a body become accredited to monitor a code of conduct?"
# user_content = "What responsibilities does an accredited body have when a code of conduct is breached?"
# user_content = "What actions can an accredited monitoring body take if a code of conduct is infringed?"
# user_content = "Under what circumstances can an accreditation be revoked?"

#user_content = "Can you give me a worfklow for the approval of a code of conduct?"

# territorial scope
user_content = "When does GDPR apply to non-EU entities?"
user_content = "I am not based in the EU. Does GDPR apply to me?"

# video
user_content = "Is video surveillance regulated under gdpr?"

# Covid Health
user_content = "What specific purposes allow healthcare data to be processed without explicit consent?"
user_content = "What are the permissible legal bases for processing health data for scientific research?"

# covid location
user_content = "What types of data should not be collected by contact tracing applications?"

# Concent
user_content = "Can children consent to data collection?"

# Forgotten
user_content = "What is the right to erasure and when does it apply?"


In [1]:
from gdpr_rag.documents.dpo import DPO
from gdpr_rag.documents.consent import Consent
dpo = DPO()
consent = Consent()

In [23]:
file = "./inputs/index/video.parquet"
df = pd.read_parquet(file)
df

,section_reference,text,source,embedding,document
0,2.1,gAAAAABmUH9YDpyn75A6P5UBv2tn3enDzLAPpQYXhXxsuk...,summary,"[-0.05341560021042824, 0.0171151515096426, -0....",Video
1,2.2,gAAAAABmUH9YEm2qVPRmHM-J5GQxVp8K7JCYlxzpbBAt6h...,summary,"[-0.05042532831430435, -0.034690551459789276, ...",Video
2,2.3,gAAAAABmUH9Yt_9fPYROaTi2ZIdE-IGpQpFtdHQ1tI6IpM...,summary,"[-0.05431677773594856, 0.031087687239050865, -...",Video
3,3,gAAAAABmUH9YI-aMX31OE0KoFJtf5__giOrAD1XTEiVt1q...,summary,"[0.011479574255645275, -0.04576420783996582, -...",Video
4,3.1.1,gAAAAABmUH9YMHnezbumcNuKAdDdVAelDbD5cu0VuRpGAz...,summary,"[0.024606412276625633, -0.012803575955331326, ...",Video
...,...,...,...,...,...
123,9.3.2,gAAAAABmUH9YxhYBbjnCjhfyzJSnOqOND6Egd3ErKrbHPX...,question,"[-0.004779711831361055, 0.0069802324287593365,...",Video
124,9.3.2,gAAAAABmUH9YaTg645WLz3LZdFQ_zBII1t0IXvpW96hTbw...,question,"[-0.045810651034116745, 0.048128221184015274, ...",Video
125,9.3.2,gAAAAABmUH9YSxCyP9WC7aFGTLMJSCrcL84pCwxTkSPlhz...,question,"[-0.02516673319041729, -0.024208365008234978, ...",Video
126,10,gAAAAABmUH9YJjSfd9EKsOg7w3xxvlvNGZ5J0T1SWUEaug...,question,"[-0.039777971804142, -0.014415937475860119, -0...",Video
